# Construyendo un sistema de análisis de sentimiento (III)

## Tarea 7a

## Enunciado

- Ejercicio 1. Cambia el tokenizador que has usado hasta ahora por el orientado a Twitter, realiza las adaptaciones que consideres y prueba.


- Ejercicio 2. Caracterización gramatical. Incorpore como característica una nueva columna por cada categoría gramatical y cuente las apariciones en cada documento. 


- Ejercicio 3. Filtrado gramatical. ¿Qué pasa si mi 'bag of words' solo considera:

    3.1 nombres

    3.2 adjetivos

    3.3 adverbios

    3.4 nombres y adjetivos

    3.5 adjetivos y adverbios

    3.6 nombres y adverbios

    3.7 nombres, adjetivos y adverbios





## Implementación

### Paso 1: Descargando las librerías de nltk

Importamos la librería nltk. Si no la tenemos instalada la podemos instalar mediante pip

    pip install nltk
 
Una vez importada, descargamos los paquetes necesarios: en este caso, con los conjuntos "popular" y "spanish grammars" será suficiente. El "popular" descargará los paquetes mas populares, mientras que "spanish grammars" descargará los paquetes para realizar el procesamiento de texto en español

In [1]:
import nltk


nltk.download('popular')
nltk.download('spanish_grammars')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk

True

### Paso 2: Cargando el corpus

Cargamos el corpus de entrenamiento desde el fichero "train_es.tsv", y el corpus de prueba desde el fichero "trial_es.tsv". Este fichero utiliza un formato csv separado por tabulaciones. Como columnas cargamos "text", correspondiente al texto del tuit; y "HS", correspondiente a las etiquetas binarias de odio/no_odio

Para ello utilizaremos la librería pandas, con el método `read_csv()`, indicando en sus parámetros el tipo de separador y el número de filas a leer. Esto nos cargará las 10 filas en un dataframe (al que llamaremos `corpus_df`) con las mismas columnas que en el fichero original, etiquetadas por sus respectivos nombres. 

Una vez cargado, mostraremos el dataframe para comprobar que se ha cargado correctamente

In [11]:
import pandas as pd

corpus_df_train = pd.read_csv("HateEval/train_es.tsv", sep="\t", usecols =["text","HS"])
labels_train = corpus_df_train[['HS']]
print(labels_train)


corpus_df_eval = pd.read_csv("HateEval/trial_es.tsv", sep="\t", usecols =["text", "HS"])
labels_eval = pd.read_csv("HateEval/trial_es.tsv", delimiter='\t', usecols=["HS"])

print(corpus_df_train)
print(corpus_df_eval)

      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467   1
4468   0

[4469 rows x 1 columns]
                                                   text  HS
0     Easyjet quiere duplicar el número de mujeres p...   1
1     El gobierno debe crear un control estricto de ...   1
2     Yo veo a mujeres destruidas por acoso laboral ...   0
3     — Yo soy respetuoso con los demás, sólamente l...   0
4     Antonio Caballero y como ser de mal gusto e ig...   0
...                                                 ...  ..
4464  @miriaan_ac @Linaveso_2105 @HumildesSquad_ CÁL...   1
4465  @IvanDuque presidente en Cúcuta , tenemos prob...   1
4466              - Callaté Visto Que Te Dejo En Puta🎤🎶   0
4467  -¿porque los hombres se casan con las mujeres?...   1
4468  — No hay nada más lento que un caracol. — Cáll...   0

[4469 rows x 2 columns]
                                                 text  HS
0   @ian_delaCalva @IrantzuVarela @pikaramagazine ...   0
1   NI

### Paso 3: Separando el texto en tokens y clasificando

En este paso, realizamos la clasificación de las palabras en función de su categoría gramatical.

Antes de realizar la clasificación, separamos el texto de cada tuit en tokens. Para ello, utilizamos la clase `TweetTokenizer`, especializada en la extracción de tokens de tuits. Con esto, transformaremos en campo `'text'` de cada tuit en una lista de tokens, lo cual nos facilitará dicha clasificación

Tras esto, pasaremos cada token a minúsculas, eliminaremos stopwords y tokens no alfanuméricos (hashtag, citas, enlaces...) y los añadiremos a una lista de palabras.


Finalmente, aplicaremos el POS Tagger, que generará una columna añadiendo la categoría gramatical de cada palabra

In [12]:
from nltk.tokenize import TweetTokenizer
from nltk.tag.stanford import StanfordPOSTagger
from nltk.corpus import stopwords

stopwords.words('spanish')

spanish_postagger = StanfordPOSTagger('./StanfordTagger/spanish.tagger', './StanfordTagger/stanford-postagger.jar', encoding='utf8')
cleantokens = []

tweet_tokenizer = TweetTokenizer()

for index, tweet in corpus_df_eval.iterrows():
    for word in tweet_tokenizer.tokenize(tweet['text']):
        word = word.lower()
        if word.isalnum() and word not in stopwords.words('spanish'):
            cleantokens.append(word)
tagged_words = spanish_postagger.tag(cleantokens)
for (word, tag) in tagged_words:
        print(word+' '+tag)
        
print(tagged_words[0][0])

oye i
molestas aq0000
puta nc0s000
madre nc0s000
ninguna di0000
mujer nc0s000
puta aq0000
editar vmn0000
además rg
complicado aq0000
hace vmip000
merezca vmsp000
pena nc0s000
puedes vmip000
haber van0000
grabado vmp0000
puta aq0000
hostia nc0s000
planos nc0p000
si cs
montas vmip000
bien rg
maquillas nc0p000
atractivamente rg
va vmip000
mierda nc0s000
bien cc
joder vmn0000
puta nc0s000
alegría nc0s000
mereces aq0000
pequeña aq0000
política aq0000
levanta vmip000
sesión nc0s000
hijos nc0p000
puta aq0000
mandan vmip000
peones nc0p000
plaza nc0s000
provocar vmn0000
violencia nc0s000
15 z0
policías nc0p000
heridos aq0000
pasa vmip000
si cs
ahí rg
pueblo nc0s000
dios vmis000
mio dp0000
pueblo nc0s000
vieja aq0000
rompe vmip000
lomo nc0s000
laburando vmg0000
casuso vmis000
callate nc0s000
puta aq0000
vez nc0s000
culpable aq0000
perra np00000
v sp000
puedo vmip000
creer vmn0000
propias aq0000
mujeres nc0p000
naturalicen vmip000
violación nc0s000
acoso nc0s000
justifiquen aq0000
mujer nc0s000
p

## Ejercicio 2: Caracterización gramatical

### Paso 1: Obtener tokens del dataframe

De nuevo, tokenizamos el campo "text" del dataframe, transformando dicha columna en una lista de palabras de cada tuit. Repetimos el mismo proceso que en el ejercicio anterior, aplicando el `TweetTokenizer()`, pasando palabras a minúsculas, y eliminando stopwords y palabras con caracteres no alfanuméricos

Aplicamos el proceso con ambos conjuntos: de entrenamiento y de prueba

In [13]:
def token_split(wordlist: pd.DataFrame):
    token_list = tweet_tokenizer.tokenize(wordlist)
    
    words_no_stop = [word.lower() for word in token_list if word not in stopwords.words('spanish') and word.isalnum()]
    return words_no_stop

corpus_filtered_train = corpus_df_train.copy()
corpus_filtered_train['text'] = corpus_df_train['text'].apply(token_split)

corpus_filtered_eval = corpus_df_eval.copy()
corpus_filtered_eval['text'] = corpus_df_eval['text'].apply(token_split)

print(corpus_filtered_train)
print(corpus_filtered_eval)

                                                   text  HS
0     [easyjet, quiere, duplicar, número, mujeres, p...   1
1     [el, gobierno, debe, crear, control, estricto,...   1
2     [yo, veo, mujeres, destruidas, acoso, laboral,...   0
3     [yo, respetuoso, demás, sólamente, recuerdo, y...   0
4     [antonio, caballero, ser, mal, gusto, ignorant...   0
...                                                 ...  ..
4464                          [cállateeee, zorra, ahre]   1
4465  [presidente, cúcuta, problemas, venezolanos, d...   1
4466          [callaté, visto, que, te, dejo, en, puta]   0
4467  [hombres, casan, mujeres, cabras, saben, frega...   1
4468  [no, lento, caracol, cállate, hijo, puta, dice...   0

[4469 rows x 2 columns]
                                                 text  HS
0                        [oye, molestas, puta, madre]   0
1                          [ninguna, mujer, es, puta]   0
2   [editar, además, complicado, hace, merezca, pe...   0
3   [bien, joder, puta,

### Paso 2: Cuenta el número de ocurrencias de cada categoría

Tras separar los tokens y filtrar las palabras, obtenemos un dataframe cuyo campo "text" se corresponde a una lista de palabras.

Revisando la documentación de estos enlaces: [consulta](https://stackoverflow.com/questions/27047450/meaning-of-stanford-spanish-pos-tagger-tags) y [documentacion sobre etiquetas](https://web.archive.org/web/20160325024315/http://nlp.lsi.upc.edu/freeling/doc/tagsets/tagset-es.html), obtenemos la forma en que StanfordTags representa las etiquetas, y la categoría gramatical de cada una. En concreto, observamos que la categoría gramatical se almacena en la primera posición de la etiqueta

Para facilitar el proceso de identificación, nos creamos un diccionario que asocie cada código con su categoría gramatical (por ejemplo: 'a' = "Adjetivo"). Tras esto, añadimos a los dataframe de los corpus una columna por cada categoría, y las inicializamos a 0.

Finalmente, nos creamos una función que itere sobre las filas, obtenga los tags de cada tuit, y actualice las columnas con el recuento de cada categoría

In [14]:
category_map = {"a": "Adjetivo", "r": "Adverbio", "d": "Determinante", "n": "Nombre", "v": "Verbo", 
                  "p": "Pronombre", "c": "Conjunción", "z": "Numeral", "s": "Adposicion"}

corpus_tagged_eval = corpus_filtered_eval.copy()
corpus_tagged_train = corpus_filtered_train.copy()

for category in category_map.values():
    corpus_tagged_eval[category] = 0
    corpus_tagged_train[category] = 0

from collections import Counter
    
def categorize_tweet(row: pd.DataFrame, axis=1):
    tagged_words = spanish_postagger.tag(row['text'])
    tags = [elem[1][0] for elem in tagged_words]
    
    num_ocurrences = Counter(tags)
    
    for tag, category in category_map.items():
        row[category] = num_ocurrences[tag]
        
    return row

corpus_tagged_eval = corpus_tagged_eval.apply(categorize_tweet, axis=1)
#corpus_tagged_train = corpus_tagged_train.apply(categorize_tweet, axis=1)

print(corpus_tagged_eval)
#print(corpus_tagged_train)

                                                 text  HS  Adjetivo  Adverbio  \
0                        [oye, molestas, puta, madre]   0         1         0   
1                          [ninguna, mujer, es, puta]   0         1         0   
2   [editar, además, complicado, hace, merezca, pe...   0         2         3   
3   [bien, joder, puta, alegría, te, mereces, pequ...   0         1         0   
4   [todo, política, levanta, sesión, hijos, puta,...   0         3         1   
..                                                ...  ..       ...       ...   
95  [si, alguien, imagen, pokemon, bailando, machu...   0         2         1   
96  [el, ciudadano, ruso, denis, yakovlev, facilit...   0         7         1   
97  [toy, feliz, voy, comer, 12, empanadas, árabes...   0         3         0   
98  [los, barrios, pueblo, solidario, caos, acogid...   0         2         0   
99  [viernes, noche, moritos, reggaeton, arabe, de...   0         1         0   

    Determinante  Nombre  V

## Ejercicio 3: Filtrado gramatical

### Filtrado de categorías gramaticales

En este ejercicio, debemos filtrar varias categorías gramaticales del texto.
Para ello, nos vamos a crear la función `filter_categories` que permitirá filtrar las palabras que se correspondan a las categorías gramaticales indicadas por parámetro. Estas se indican mediante su código ("a" = Adjetivos, "n" = Nombres... etc)

In [21]:
def filter_categories(row: pd.DataFrame, category_list: list):
    tagged_words = spanish_postagger.tag(row['text'])    
    tokens_filtered = [elem[0] for elem in tagged_words if elem[1][0] in category_list]
    
    row['text'] = tokens_filtered
        
    return row

#### 3.1. Filtrado de nombres

Para filtrar los nombres, aplicamos la función anterior indicando el código "n" dentro de la lista de categorías

In [22]:
corpus_onlynames_eval = corpus_filtered_eval.copy()
corpus_onlynames_train = corpus_filtered_train.copy()

corpus_onlynames_eval = corpus_onlynames_eval.apply(filter_categories, args=["n"], axis=1)
corpus_onlynames_train = corpus_onlynames_train.apply(filter_categories, args=["n"], axis=1)

print(corpus_onlynames_eval)
print(corpus_onlynames_train)

                                                 text  HS
0                                       [puta, madre]   0
1                                             [mujer]   0
2           [pena, hostia, planos, maquillas, mierda]   0
3                                     [puta, alegría]   0
4   [política, sesión, hijos, peones, plaza, viole...   0
..                                                ...  ..
95                       [imagen, machupichu, cobras]   0
96        [ciudadano, yakovlev, matrimonios, mayoría]   0
97                                   [toy, empanadas]   0
98  [barrios, pueblo, caos, acogida, pabellones, vía]   0
99              [viernes, noche, reggaeton, forniteo]   0

[100 rows x 2 columns]
                                                   text  HS
0             [easyjet, número, mujeres, piloto, avión]   1
1         [gobierno, control, inmigración, zonas, masa]   1
2     [mujeres, acoso, depresión, violación, maltrat...   0
3                                      [

#### 3.2. Filtrado de adjetivos

In [23]:
corpus_onlyadjectives_eval = corpus_filtered_eval.copy()
corpus_onlyadjectives_train = corpus_filtered_train.copy()

corpus_onlyadjectives_eval = corpus_onlyadjectives_eval.apply(filter_categories, args=["a"], axis=1)
corpus_onlyadjectives_train = corpus_onlyadjectives_train.apply(filter_categories, args=["a"], axis=1)

print(corpus_onlyadjectives_eval)
print(corpus_onlyadjectives_train)

                                                 text  HS
0                                          [molestas]   0
1                                              [puta]   0
2                                  [complicado, puta]   0
3                                           [pequeña]   0
4                              [puta, heridos, vieja]   0
..                                                ...  ..
95                  [pokemon, tirititirititiritiirii]   0
96  [ruso, florida, falsos, estadounidenses, inmig...   0
97                            [feliz, árabes, naruto]   0
98                           [solidario, inmigrantes]   0
99                                          [moritos]   0

[100 rows x 2 columns]
                                                   text  HS
0                                                    []   1
1                     [estricto, fronterizas, colombia]   1
2     [destruidas, laboral, callejero, sexual, físic...   0
3                                       

#### 3.3. Filtrado de adverbios

In [24]:
corpus_onlyadverbs_eval = corpus_filtered_eval.copy()
corpus_onlyadverbs_train = corpus_filtered_train.copy()

corpus_onlyadverbs_eval = corpus_onlyadverbs_eval.apply(filter_categories, args=["r"], axis=1)
corpus_onlyadverbs_train = corpus_onlyadverbs_train.apply(filter_categories, args=["r"], axis=1)

print(corpus_onlyadverbs_eval)
print(corpus_onlyadverbs_train)

                              text  HS
0                               []   0
1                               []   0
2   [además, bien, atractivamente]   0
3                               []   0
4                            [ahí]   0
..                             ...  ..
95                           [hoy]   0
96                         [menos]   0
97                              []   0
98                              []   0
99                              []   0

[100 rows x 2 columns]
                    text  HS
0                     []   1
1              [después]   1
2                     []   0
3     [sólamente, claro]   0
4                     []   0
...                  ...  ..
4464                  []   1
4465           [no, acá]   1
4466                  []   0
4467                  []   1
4468         [no, lento]   0

[4469 rows x 2 columns]


#### 3.4. Filtrado de nombres y adjetivos

In [29]:
corpus_nameadj_eval = corpus_filtered_eval.copy()
corpus_nameadj_train = corpus_filtered_train.copy()

corpus_nameadj_eval = corpus_nameadj_eval.apply(filter_categories, args=[["n", "a"]], axis=1)
corpus_nameadj_train = corpus_nameadj_train.apply(filter_categories, args=[["n", "a"]], axis=1)

print(corpus_nameadj_eval)
print(corpus_nameadj_train)

                                                 text  HS
0                             [molestas, puta, madre]   0
1                                       [mujer, puta]   0
2   [complicado, pena, puta, hostia, planos, maqui...   0
3                            [puta, alegría, pequeña]   0
4   [política, sesión, hijos, puta, peones, plaza,...   0
..                                                ...  ..
95  [imagen, pokemon, machupichu, cobras, tirititi...   0
96  [ciudadano, ruso, yakovlev, florida, matrimoni...   0
97            [toy, feliz, empanadas, árabes, naruto]   0
98  [barrios, pueblo, solidario, caos, acogida, in...   0
99     [viernes, noche, moritos, reggaeton, forniteo]   0

[100 rows x 2 columns]
                                                   text  HS
0             [easyjet, número, mujeres, piloto, avión]   1
1     [gobierno, control, estricto, inmigración, zon...   1
2     [mujeres, destruidas, acoso, laboral, callejer...   0
3                          [respetuoso, 

#### 3.5. Filtrado de adjetivos y adverbios

In [30]:
corpus_adjadv_eval = corpus_filtered_eval.copy()
corpus_adjadv_train = corpus_filtered_train.copy()

corpus_adjadv_eval = corpus_adjadv_eval.apply(filter_categories, args=[["a","r"]], axis=1)
corpus_adjadv_train = corpus_adjadv_train.apply(filter_categories, args=[["a","r"]], axis=1)

print(corpus_adjadv_eval)
print(corpus_adjadv_train)

                                                 text  HS
0                                          [molestas]   0
1                                              [puta]   0
2    [además, complicado, puta, bien, atractivamente]   0
3                                           [pequeña]   0
4                         [puta, heridos, ahí, vieja]   0
..                                                ...  ..
95             [pokemon, hoy, tirititirititiritiirii]   0
96  [ruso, florida, menos, falsos, estadounidenses...   0
97                            [feliz, árabes, naruto]   0
98                           [solidario, inmigrantes]   0
99                                          [moritos]   0

[100 rows x 2 columns]
                                                   text  HS
0                                                    []   1
1            [estricto, fronterizas, colombia, después]   1
2     [destruidas, laboral, callejero, sexual, físic...   0
3                        [respetuoso, só

#### 3.6. Filtrado de nombres y adverbios

In [31]:
corpus_nameadv_eval = corpus_filtered_eval.copy()
corpus_nameadv_train = corpus_filtered_train.copy()

corpus_nameadv_eval = corpus_nameadv_eval.apply(filter_categories, args=[["n","r"]], axis=1)
corpus_nameadv_train = corpus_nameadv_train.apply(filter_categories, args=[["n","r"]], axis=1)

print(corpus_nameadv_eval)
print(corpus_nameadv_train)

                                                 text  HS
0                                       [puta, madre]   0
1                                             [mujer]   0
2   [además, pena, hostia, planos, bien, maquillas...   0
3                                     [puta, alegría]   0
4   [política, sesión, hijos, peones, plaza, viole...   0
..                                                ...  ..
95                  [imagen, machupichu, hoy, cobras]   0
96  [ciudadano, yakovlev, menos, matrimonios, mayo...   0
97                                   [toy, empanadas]   0
98  [barrios, pueblo, caos, acogida, pabellones, vía]   0
99              [viernes, noche, reggaeton, forniteo]   0

[100 rows x 2 columns]
                                                   text  HS
0             [easyjet, número, mujeres, piloto, avión]   1
1     [gobierno, control, inmigración, zonas, despué...   1
2     [mujeres, acoso, depresión, violación, maltrat...   0
3                    [sólamente, escoria

#### 3.7. Filtrado de nombres, adjetivos y adverbios

In [51]:
corpus_nameadjadv_eval = corpus_filtered_eval.copy()
corpus_nameadjadv_train = corpus_filtered_train.copy()

corpus_nameadjadv_eval = corpus_nameadjadv_eval.apply(filter_categories, args=[["n","a","r"]], axis=1)
corpus_nameadjadv_train = corpus_nameadjadv_train.apply(filter_categories, args=[["n","a","r"]], axis=1)

print(corpus_nameadjadv_eval)
print(corpus_nameadjadv_train)

                                                 text  HS
0                             [molestas, puta, madre]   0
1                                       [mujer, puta]   0
2   [además, complicado, pena, puta, hostia, plano...   0
3                            [puta, alegría, pequeña]   0
4   [política, sesión, hijos, puta, peones, plaza,...   0
..                                                ...  ..
95  [imagen, pokemon, machupichu, hoy, cobras, tir...   0
96  [ciudadano, ruso, yakovlev, florida, menos, ma...   0
97            [toy, feliz, empanadas, árabes, naruto]   0
98  [barrios, pueblo, solidario, caos, acogida, in...   0
99     [viernes, noche, moritos, reggaeton, forniteo]   0

[100 rows x 2 columns]
                                                   text  HS
0             [easyjet, número, mujeres, piloto, avión]   1
1     [gobierno, control, estricto, inmigración, zon...   1
2     [mujeres, destruidas, acoso, laboral, callejer...   0
3        [respetuoso, sólamente, escoria

## Preparando la bolsa de palabras

### 3.1. Generando bolsa de nombres 

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_onlynames_str_train = corpus_onlynames_train.copy()
corpus_onlynames_str_eval = corpus_onlynames_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_onlynames_str_train["text"] = corpus_onlynames_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_onlynames_str_eval["text"] = corpus_onlynames_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
onlynames_vectorizer_train = CountVectorizer()
onlynames_vectorizer_eval = CountVectorizer()

# construimos vocabulario
vector_onlynames_train = onlynames_vectorizer_train.fit_transform(corpus_onlynames_str_train["text"])
vector_onlynames_eval = onlynames_vectorizer_eval.fit_transform(corpus_onlynames_str_eval["text"])

print(onlynames_vectorizer_train.vocabulary_)
print(onlynames_vectorizer_eval.vocabulary_)

{'easyjet': 1788, 'número': 3852, 'mujeres': 3697, 'piloto': 4286, 'avión': 424, 'gobierno': 2411, 'control': 1325, 'inmigración': 2828, 'zonas': 5819, 'masa': 3384, 'acoso': 59, 'depresión': 1579, 'violación': 5645, 'maltrato': 3290, 'tipo': 5314, 'comportamientos': 1240, 'show': 5023, 'escoria': 1962, 'tomas': 5338, 'antonio': 268, 'caballero': 719, 'gusto': 2507, 'conductas': 1276, 'violencia': 5650, 'hijo': 2617, 'puta': 4594, 'sobra': 5069, 'mundo': 3706, 'pablo': 3965, 'máster': 3724, 'pego': 4152, 'patada': 4092, 'motocicleta': 3668, 'haitiano': 2550, 'zorra': 5822, 'padres': 3973, 'huelga': 2697, 'hambre': 2553, 'personas': 4234, 'inmigrantes': 2830, 'delincuentes': 1556, 'tomi': 5341, 'duelo': 1769, 'gallina': 2332, 'cntala': 1149, 'bolso': 627, 'culo': 1446, 'albondigas': 151, 'estilo': 2019, 'fodongas': 2229, 'presidente': 4457, 'lujo': 3208, 'mundial': 3705, 'ilusos': 2761, 'juez': 2991, 'abusos': 45, 'parte': 4063, 'vamoooo': 5544, 'madre': 3238, 'esfuerzo': 1974, 'teléfon

### 3.2. Generando bolsa de adjetivos

In [60]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_onlyadjectives_str_train = corpus_onlyadjectives_train.copy()
corpus_onlyadjectives_str_eval = corpus_onlyadjectives_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_onlyadjectives_str_train["text"] = corpus_onlyadjectives_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_onlyadjectives_str_eval["text"] = corpus_onlyadjectives_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
onlyadjectives_vectorizer_train = CountVectorizer()
onlyadjectives_vectorizer_eval = CountVectorizer()

# construimos vocabulario
vector_onlyadjectives_train = onlyadjectives_vectorizer_train.fit_transform(corpus_onlyadjectives_str_train["text"])
vector_onlyadjectives_eval = onlyadjectives_vectorizer_eval.fit_transform(corpus_onlyadjectives_str_eval["text"])

print(onlyadjectives_vectorizer_train.vocabulary_)
print(onlyadjectives_vectorizer_eval.vocabulary_)

{'estricto': 1220, 'fronterizas': 1423, 'colombia': 696, 'destruidas': 974, 'laboral': 1936, 'callejero': 512, 'sexual': 3200, 'físico': 1448, 'machistas': 2058, 'pobre': 2720, 'respetuoso': 3009, 'mal': 2081, 'ignorante': 1658, 'claro': 670, 'casado': 556, 'cara': 533, 'guatapanal': 1549, 'nacional': 2356, 'indocumentado': 1734, 'indocumentados': 1735, 'alegre': 128, 'cállate': 852, 'puta': 2875, 'facil': 1286, 'hijooo': 1606, 'gabacho': 1452, 'perú': 2666, 'menor': 2189, 'jugadores': 1913, 'solo': 3258, 'trabajador': 3439, 'inmigrante': 1776, 'fundido': 1442, 'difícil': 1003, 'sevillista': 3197, 'pepe': 2625, 'moralinas': 2288, 'neta': 2397, 'divertido': 1029, 'metido': 2218, 'viejos': 3607, 'mereces': 2204, 'tartaja': 3363, 'madeira': 2062, 'gordo': 1507, 'almagro': 148, 'gran': 1514, 'perico': 2642, 'gemelas': 1478, 'verdadera': 3573, 'argentos': 262, 'sudakas': 3310, 'miserables': 2255, 'escorias': 1174, 'mico': 2228, 'sorete': 3268, 'parido': 2550, 'video': 3602, 'norteño': 2422,

### 3.3. Generando bolsa de adverbios

In [62]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_onlyadverbs_str_train = corpus_onlyadverbs_train.copy()
corpus_onlyadverbs_str_eval = corpus_onlyadverbs_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_onlyadverbs_str_train["text"] = corpus_onlyadverbs_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_onlyadverbs_str_eval["text"] = corpus_onlyadverbs_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
onlyadverbs_vectorizer_train = CountVectorizer()
onlyadverbs_vectorizer_eval = CountVectorizer()

# construimos vocabulario
vector_onlyadverbs_train = onlyadverbs_vectorizer_train.fit_transform(corpus_onlyadverbs_str_train["text"])
vector_onlyadverbs_eval = onlyadverbs_vectorizer_eval.fit_transform(corpus_onlyadverbs_str_eval["text"])

print(onlyadverbs_vectorizer_train.vocabulary_)
print(onlyadverbs_vectorizer_eval.vocabulary_)

{'después': 98, 'sólamente': 304, 'claro': 74, 'no': 232, 'jakskksjsj': 180, 'jajajaaj': 174, 'casi': 69, 'tanto': 309, 'todavía': 317, 'mas': 207, 'si': 288, 'mal': 205, 'demasiado': 93, 'aquí': 49, 'muchísimo': 225, 'almas': 34, 'asi': 52, 'nada': 229, 'ahora': 23, 'así': 53, 'atrás': 57, 'delante': 92, 'acá': 7, 'ya': 329, 'dentro': 95, 'super': 301, 'aqui': 48, 'bien': 65, 'alrededor': 35, 'luego': 204, 'enseñarle': 121, 'polla': 257, 'encima': 111, 'ahorita': 24, 'justo': 188, 'tan': 308, 'mucho': 224, 'ayer': 61, 'hoy': 157, 'emo': 108, 'siempre': 289, 'peor': 247, 'mañana': 209, 'ahí': 26, 'nunca': 238, 'seguramente': 284, 'bastante': 64, 'despacio': 96, 'indirectamente': 166, 're': 274, 'solamente': 297, 'damián': 86, 'primero': 266, 'solo': 298, 'eva': 137, 'culturalmente': 84, 'ni': 231, 'mejor': 213, 'tarde': 310, 'ahi': 22, 'nomas': 233, 'política': 258, 'completamente': 75, 'sólo': 305, 'jamás': 182, 'alá': 37, 'mañas': 210, 'indiscriminadamente': 167, 'cómo': 85, 'enseñar

### 3.4. Generando bolsa de nombres y adjetivos

In [63]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_nameadj_str_train = corpus_nameadj_train.copy()
corpus_nameadj_str_eval = corpus_nameadj_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_nameadj_str_train["text"] = corpus_nameadj_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_nameadj_str_eval["text"] = corpus_nameadj_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
nameadj_vectorizer_train = CountVectorizer()
nameadj_vectorizer_eval = CountVectorizer()

# construimos vocabulario
vector_nameadj_train = nameadj_vectorizer_train.fit_transform(corpus_nameadj_str_train["text"])
vector_nameadj_eval = nameadj_vectorizer_eval.fit_transform(corpus_nameadj_str_eval["text"])

print(nameadj_vectorizer_train.vocabulary_)
print(nameadj_vectorizer_eval.vocabulary_)

{'easyjet': 2729, 'número': 5949, 'mujeres': 5701, 'piloto': 6594, 'avión': 732, 'gobierno': 3716, 'control': 1998, 'estricto': 3094, 'inmigración': 4377, 'zonas': 8990, 'fronterizas': 3529, 'colombia': 1796, 'masa': 5243, 'destruidas': 2491, 'acoso': 106, 'laboral': 4717, 'callejero': 1243, 'depresión': 2388, 'violación': 8756, 'sexual': 7770, 'maltrato': 5117, 'físico': 3582, 'tipo': 8265, 'comportamientos': 1871, 'machistas': 5019, 'show': 7794, 'pobre': 6699, 'respetuoso': 7387, 'escoria': 2989, 'tomas': 8300, 'antonio': 468, 'caballero': 1146, 'mal': 5080, 'gusto': 3867, 'ignorante': 4191, 'claro': 1715, 'conductas': 1917, 'violencia': 8762, 'hijo': 4013, 'puta': 7065, 'sobra': 7870, 'mundo': 5713, 'pablo': 6131, 'casado': 1413, 'máster': 5740, 'pego': 6384, 'patada': 6298, 'cara': 1344, 'guatapanal': 3836, 'motocicleta': 5654, 'nacional': 5763, 'haitiano': 3923, 'indocumentado': 4311, 'zorra': 8994, 'padres': 6142, 'indocumentados': 4312, 'huelga': 4119, 'hambre': 3926, 'personas

### 3.5. Generando bolsa de adjetivos y adverbios

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_adjadv_str_train = corpus_adjadv_train.copy()
corpus_adjadv_str_eval = corpus_adjadv_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_adjadv_str_train["text"] = corpus_adjadv_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_adjadv_str_eval["text"] = corpus_adjadv_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
adjadv_vectorizer_train = CountVectorizer()
adjadv_vectorizer_eval = CountVectorizer()

# construimos vocabulario
vector_adjadv_train = adjadv_vectorizer_train.fit_transform(corpus_adjadv_str_train["text"])
vector_adjadv_eval = adjadv_vectorizer_eval.fit_transform(corpus_adjadv_str_eval["text"])

print(adjadv_vectorizer_train.vocabulary_)
print(adjadv_vectorizer_eval.vocabulary_)

{'estricto': 1346, 'fronterizas': 1561, 'colombia': 761, 'después': 1057, 'destruidas': 1062, 'laboral': 2108, 'callejero': 572, 'sexual': 3452, 'físico': 1587, 'machistas': 2241, 'pobre': 2947, 'respetuoso': 3255, 'sólamente': 3609, 'claro': 735, 'mal': 2264, 'ignorante': 1803, 'casado': 616, 'cara': 593, 'guatapanal': 1689, 'nacional': 2558, 'indocumentado': 1887, 'indocumentados': 1888, 'no': 2616, 'alegre': 155, 'jakskksjsj': 2052, 'cállate': 926, 'puta': 3113, 'facil': 1417, 'jajajaaj': 2035, 'hijooo': 1747, 'casi': 619, 'gabacho': 1591, 'perú': 2889, 'menor': 2377, 'jugadores': 2083, 'tanto': 3627, 'todavía': 3709, 'solo': 3521, 'trabajador': 3722, 'inmigrante': 1930, 'fundido': 1581, 'difícil': 1093, 'sevillista': 3449, 'mas': 2330, 'pepe': 2845, 'si': 3463, 'moralinas': 2482, 'neta': 2601, 'divertido': 1121, 'metido': 2408, 'viejos': 3896, 'mereces': 2394, 'demasiado': 995, 'tartaja': 3637, 'madeira': 2245, 'gordo': 1646, 'almagro': 179, 'gran': 1654, 'aquí': 293, 'muchísimo': 

### 3.6. Generando bolsa de nombres y adverbios

In [65]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_nameadv_str_train = corpus_nameadv_train.copy()
corpus_nameadv_str_eval = corpus_nameadv_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_nameadv_str_train["text"] = corpus_nameadv_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_nameadv_str_eval["text"] = corpus_nameadv_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
nameadv_vectorizer_train = CountVectorizer()
nameadv_vectorizer_eval = CountVectorizer()

# construimos vocabulario
vector_nameadv_train = nameadv_vectorizer_train.fit_transform(corpus_nameadv_str_train["text"])
vector_nameadv_eval = nameadv_vectorizer_eval.fit_transform(corpus_nameadv_str_eval["text"])

print(nameadv_vectorizer_train.vocabulary_)
print(nameadv_vectorizer_eval.vocabulary_)

{'easyjet': 1881, 'número': 4066, 'mujeres': 3899, 'piloto': 4511, 'avión': 478, 'gobierno': 2547, 'control': 1395, 'inmigración': 2980, 'zonas': 6105, 'después': 1713, 'masa': 3570, 'acoso': 65, 'depresión': 1665, 'violación': 5929, 'maltrato': 3475, 'tipo': 5589, 'comportamientos': 1308, 'show': 5275, 'sólamente': 5458, 'escoria': 2085, 'claro': 1197, 'tomas': 5616, 'antonio': 305, 'caballero': 780, 'gusto': 2645, 'conductas': 1345, 'violencia': 5934, 'hijo': 2756, 'puta': 4833, 'sobra': 5328, 'mundo': 3909, 'pablo': 4183, 'máster': 3929, 'pego': 4373, 'patada': 4313, 'motocicleta': 3865, 'haitiano': 2688, 'zorra': 6108, 'padres': 4192, 'huelga': 2839, 'hambre': 2691, 'no': 4009, 'personas': 4459, 'inmigrantes': 2982, 'delincuentes': 1639, 'tomi': 5619, 'jakskksjsj': 3087, 'duelo': 1861, 'gallina': 2468, 'cntala': 1216, 'bolso': 687, 'culo': 1520, 'jajajaaj': 3070, 'albondigas': 178, 'casi': 989, 'estilo': 2143, 'fodongas': 2361, 'presidente': 4691, 'lujo': 3393, 'mundial': 3907, 'il

### 3.7. Generando bolsa de nombres, adjetivos y adverbios

In [66]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_nameadjadv_str_train = corpus_nameadjadv_train.copy()
corpus_nameadjadv_str_eval = corpus_nameadjadv_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_nameadjadv_str_train["text"] = corpus_nameadjadv_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_nameadjadv_str_eval["text"] = corpus_nameadjadv_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
nameadjadv_vectorizer_train = CountVectorizer()
nameadjadv_vectorizer_eval = CountVectorizer()

# construimos vocabulario
vector_nameadjadv_train = nameadjadv_vectorizer_train.fit_transform(corpus_nameadjadv_str_train["text"])
vector_nameadjadv_eval = nameadjadv_vectorizer_eval.fit_transform(corpus_nameadjadv_str_eval["text"])

print(nameadjadv_vectorizer_train.vocabulary_)
print(nameadjadv_vectorizer_eval.vocabulary_)

{'easyjet': 2814, 'número': 6145, 'mujeres': 5885, 'piloto': 6800, 'avión': 781, 'gobierno': 3844, 'control': 2060, 'estricto': 3210, 'inmigración': 4519, 'zonas': 9250, 'fronterizas': 3656, 'colombia': 1856, 'después': 2559, 'masa': 5413, 'destruidas': 2571, 'acoso': 112, 'laboral': 4875, 'callejero': 1298, 'depresión': 2466, 'violación': 9014, 'sexual': 7999, 'maltrato': 5286, 'físico': 3710, 'tipo': 8515, 'comportamientos': 1932, 'machistas': 5188, 'show': 8024, 'pobre': 6908, 'respetuoso': 7611, 'sólamente': 8319, 'escoria': 3104, 'claro': 1775, 'tomas': 8553, 'antonio': 502, 'caballero': 1201, 'mal': 5249, 'gusto': 3996, 'ignorante': 4324, 'conductas': 1978, 'violencia': 9020, 'hijo': 4143, 'puta': 7283, 'sobra': 8107, 'mundo': 5898, 'pablo': 6331, 'casado': 1468, 'máster': 5927, 'pego': 6587, 'patada': 6501, 'cara': 1399, 'guatapanal': 3965, 'motocicleta': 5833, 'nacional': 5950, 'haitiano': 4052, 'indocumentado': 4452, 'zorra': 9254, 'padres': 6343, 'indocumentados': 4453, 'huel

## Entrenando los modelos

### Paso 1: Dividiendo las colecciones en subconjuntos

#### 3.1. Subconjuntos para bolsa de nombres

In [59]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_names_train, X_names_test, y_names_train, y_names_test = train_test_split(vector_onlynames_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_names_train)
print(y_names_train)
print(labels_train)

  (0, 3697)	1
  (0, 5706)	1
  (0, 2008)	1
  (0, 3887)	1
  (0, 5357)	1
  (0, 5606)	1
  (0, 2304)	1
  (0, 5511)	1
  (0, 4949)	1
  (1, 5585)	1
  (1, 4352)	1
  (1, 566)	1
  (1, 305)	1
  (2, 5372)	2
  (2, 432)	1
  (2, 1983)	1
  (2, 4512)	1
  (2, 3158)	1
  (3, 5614)	1
  (3, 3696)	1
  (3, 2513)	1
  (3, 3629)	1
  (3, 5785)	1
  (3, 406)	1
  (4, 1954)	1
  :	:
  (3569, 499)	1
  (3569, 2420)	1
  (3570, 3599)	1
  (3570, 1785)	1
  (3570, 4816)	1
  (3570, 138)	1
  (3570, 4194)	1
  (3570, 1904)	1
  (3570, 657)	1
  (3570, 1021)	1
  (3570, 1778)	1
  (3570, 807)	1
  (3570, 1170)	1
  (3570, 2273)	1
  (3571, 1473)	1
  (3571, 3798)	1
  (3571, 3827)	1
  (3572, 2828)	1
  (3572, 5247)	1
  (3572, 1585)	1
  (3572, 4426)	1
  (3573, 216)	1
  (3574, 3706)	1
  (3574, 3521)	1
  (3574, 3213)	1
      HS
1456   0
4364   0
912    0
1461   1
3389   0
...   ..
4334   0
22     0
3028   0
2402   1
1063   1

[3575 rows x 1 columns]
      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467 

#### 3.2. Subconjuntos para bolsa de adjetivos

In [67]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_adj_train, X_adj_test, y_adj_train, y_adj_test = train_test_split(vector_onlyadjectives_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_adj_train)
print(y_adj_train)
print(labels_train)

  (0, 1777)	1
  (0, 27)	1
  (0, 8)	1
  (0, 332)	1
  (0, 911)	1
  (0, 2750)	1
  (0, 663)	1
  (1, 2875)	1
  (2, 1777)	1
  (2, 992)	1
  (3, 2875)	1
  (3, 954)	1
  (4, 2875)	1
  (4, 3293)	1
  (4, 305)	1
  (5, 1344)	1
  (5, 458)	1
  (5, 1548)	1
  (6, 1516)	1
  (6, 1610)	1
  (8, 3689)	1
  (8, 3125)	1
  (8, 2047)	1
  (9, 1566)	1
  (9, 1347)	1
  :	:
  (3562, 1983)	1
  (3562, 2421)	1
  (3562, 1778)	1
  (3563, 2882)	1
  (3565, 1596)	1
  (3565, 509)	1
  (3566, 852)	1
  (3566, 2604)	1
  (3567, 670)	1
  (3567, 3303)	1
  (3567, 1058)	1
  (3567, 945)	1
  (3568, 2875)	1
  (3568, 2204)	1
  (3568, 3455)	1
  (3569, 3334)	1
  (3569, 2993)	1
  (3570, 2440)	1
  (3570, 2536)	1
  (3570, 3361)	1
  (3571, 2625)	1
  (3572, 2782)	1
  (3573, 2440)	1
  (3574, 2875)	1
  (3574, 1224)	1
      HS
1456   0
4364   0
912    0
1461   1
3389   0
...   ..
4334   0
22     0
3028   0
2402   1
1063   1

[3575 rows x 1 columns]
      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467   1
446

#### 3.3. Subconjuntos para lista de adverbios

In [69]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_adv_train, X_adv_test, y_adv_train, y_adv_test = train_test_split(vector_onlyadverbs_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_adv_train)
print(y_adv_train)
print(labels_train)

  (0, 331)	1
  (2, 232)	1
  (2, 157)	1
  (6, 157)	1
  (8, 298)	1
  (9, 205)	1
  (9, 209)	1
  (9, 315)	1
  (10, 238)	1
  (11, 65)	1
  (12, 92)	1
  (12, 213)	1
  (13, 59)	1
  (16, 157)	1
  (18, 207)	1
  (21, 307)	1
  (26, 288)	1
  (26, 209)	1
  (33, 24)	1
  (39, 207)	1
  (39, 288)	1
  (40, 69)	1
  (42, 258)	1
  (42, 75)	1
  (43, 318)	1
  :	:
  (3542, 52)	1
  (3544, 121)	1
  (3550, 157)	1
  (3550, 307)	1
  (3550, 71)	1
  (3550, 262)	1
  (3551, 329)	1
  (3551, 298)	1
  (3552, 329)	1
  (3555, 24)	1
  (3557, 277)	1
  (3558, 69)	1
  (3558, 308)	1
  (3562, 9)	1
  (3566, 252)	1
  (3567, 120)	1
  (3569, 289)	1
  (3569, 298)	1
  (3569, 62)	1
  (3570, 232)	1
  (3571, 288)	1
  (3573, 289)	2
  (3573, 213)	1
  (3574, 49)	1
  (3574, 307)	1
      HS
1456   0
4364   0
912    0
1461   1
3389   0
...   ..
4334   0
22     0
3028   0
2402   1
1063   1

[3575 rows x 1 columns]
      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467   1
4468   0

[4469 rows x 1 columns]


#### 3.4. Generando subconjuntos para bolsa de nombres y adjetivos

In [70]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_nameadj_train, X_nameadj_test, y_nameadj_train, y_nameadj_test = train_test_split(vector_nameadj_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_nameadj_train)
print(y_nameadj_train)
print(labels_train)

  (0, 5701)	1
  (0, 4379)	1
  (0, 8853)	1
  (0, 3062)	1
  (0, 6011)	1
  (0, 8322)	1
  (0, 67)	1
  (0, 25)	1
  (0, 8711)	1
  (0, 691)	1
  (0, 2362)	1
  (0, 6764)	1
  (0, 3565)	1
  (0, 8556)	1
  (0, 1696)	1
  (0, 7666)	1
  (1, 7065)	1
  (1, 8667)	1
  (1, 6692)	1
  (1, 920)	1
  (1, 529)	1
  (2, 4379)	1
  (2, 8342)	2
  (2, 2535)	1
  (2, 3027)	1
  :	:
  (3570, 6460)	1
  (3570, 2896)	1
  (3570, 1043)	1
  (3570, 1550)	1
  (3570, 8123)	1
  (3570, 2713)	1
  (3570, 1265)	1
  (3570, 1769)	1
  (3570, 3514)	1
  (3571, 2220)	1
  (3571, 6452)	1
  (3571, 5874)	1
  (3571, 5915)	1
  (3572, 4377)	1
  (3572, 8158)	1
  (3572, 2394)	1
  (3572, 6808)	1
  (3572, 6838)	1
  (3573, 375)	1
  (3573, 5967)	1
  (3574, 7065)	1
  (3574, 5713)	1
  (3574, 5446)	1
  (3574, 3104)	1
  (3574, 4997)	1
      HS
1456   0
4364   0
912    0
1461   1
3389   0
...   ..
4334   0
22     0
3028   0
2402   1
1063   1

[3575 rows x 1 columns]
      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467

#### 3.5. Generando subconjuntos para bolsa de adjetivos y adverbios

In [71]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_adjadv_train, X_adjadv_test, y_adjadv_train, y_adjadv_test = train_test_split(vector_adjadv_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_adjadv_train)
print(y_adjadv_train)
print(labels_train)

  (0, 1931)	1
  (0, 31)	1
  (0, 11)	1
  (0, 381)	1
  (0, 993)	1
  (0, 2980)	1
  (0, 3992)	1
  (0, 727)	1
  (1, 3113)	1
  (2, 2616)	1
  (2, 1779)	1
  (2, 1931)	1
  (2, 1082)	1
  (3, 3113)	1
  (3, 1039)	1
  (4, 3113)	1
  (4, 3556)	1
  (4, 351)	1
  (5, 1477)	1
  (5, 518)	1
  (5, 1688)	1
  (6, 1779)	1
  (6, 1656)	1
  (6, 1752)	1
  (8, 3521)	1
  :	:
  (3567, 1252)	1
  (3567, 1150)	1
  (3567, 1030)	1
  (3568, 3113)	1
  (3568, 2394)	1
  (3568, 3739)	1
  (3569, 3521)	1
  (3569, 3468)	1
  (3569, 3600)	1
  (3569, 408)	1
  (3569, 3239)	1
  (3570, 2616)	1
  (3570, 2654)	1
  (3570, 2754)	1
  (3570, 3635)	1
  (3571, 2845)	1
  (3571, 3463)	1
  (3572, 3016)	1
  (3573, 2367)	1
  (3573, 3468)	2
  (3573, 2654)	1
  (3574, 3113)	1
  (3574, 293)	1
  (3574, 1350)	1
  (3574, 3624)	1
      HS
1456   0
4364   0
912    0
1461   1
3389   0
...   ..
4334   0
22     0
3028   0
2402   1
1063   1

[3575 rows x 1 columns]
      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467   

#### 3.6. Generando subconjuntos para bolsa de nombres y adverbios

In [72]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_nameadv_train, X_nameadv_test, y_nameadv_train, y_nameadv_test = train_test_split(vector_nameadv_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_nameadv_train)
print(y_nameadv_train)
print(labels_train)

  (0, 3899)	1
  (0, 5991)	1
  (0, 2132)	1
  (0, 4103)	1
  (0, 5635)	1
  (0, 5890)	1
  (0, 6142)	1
  (0, 2440)	1
  (0, 5793)	1
  (0, 5198)	1
  (1, 5869)	1
  (1, 4579)	1
  (1, 623)	1
  (1, 349)	1
  (2, 4009)	1
  (2, 5651)	2
  (2, 2826)	1
  (2, 486)	1
  (2, 2106)	1
  (2, 4748)	1
  (2, 3339)	1
  (3, 5898)	1
  (3, 3898)	1
  (3, 2651)	1
  (3, 3826)	1
  :	:
  (3570, 164)	1
  (3570, 4416)	1
  (3570, 2007)	1
  (3570, 717)	1
  (3570, 1085)	1
  (3570, 1870)	1
  (3570, 868)	1
  (3570, 1237)	1
  (3570, 2407)	1
  (3571, 1548)	1
  (3571, 4005)	1
  (3571, 5276)	1
  (3571, 4039)	1
  (3572, 2980)	1
  (3572, 5521)	1
  (3572, 1671)	1
  (3572, 4659)	1
  (3573, 251)	1
  (3573, 5279)	2
  (3573, 3621)	1
  (3574, 3909)	1
  (3574, 337)	1
  (3574, 3715)	1
  (3574, 5479)	1
  (3574, 3398)	1
      HS
1456   0
4364   0
912    0
1461   1
3389   0
...   ..
4334   0
22     0
3028   0
2402   1
1063   1

[3575 rows x 1 columns]
      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467

#### 3.7. Generando subconjuntos para bolsa de nombres, adjetivos y adverbios

In [73]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_nameadjadv_train, X_nameadjadv_test, y_nameadjadv_train, y_nameadjadv_test = train_test_split(vector_nameadjadv_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_nameadjadv_train)
print(y_nameadjadv_train)
print(labels_train)

  (0, 5885)	1
  (0, 4521)	1
  (0, 9112)	1
  (0, 3178)	1
  (0, 6209)	1
  (0, 8575)	1
  (0, 71)	1
  (0, 28)	1
  (0, 8969)	1
  (0, 736)	1
  (0, 2437)	1
  (0, 6975)	1
  (0, 9303)	1
  (0, 3693)	1
  (0, 8812)	1
  (0, 1755)	1
  (0, 7893)	1
  (1, 7283)	1
  (1, 8925)	1
  (1, 6900)	1
  (1, 972)	1
  (1, 570)	1
  (2, 6069)	1
  (2, 4521)	1
  (2, 8596)	2
  :	:
  (3570, 2797)	1
  (3570, 1320)	1
  (3570, 1829)	1
  (3570, 3640)	1
  (3571, 2287)	1
  (3571, 6655)	1
  (3571, 6063)	1
  (3571, 8025)	1
  (3571, 6109)	1
  (3572, 4519)	1
  (3572, 8408)	1
  (3572, 2472)	1
  (3572, 7021)	1
  (3572, 7052)	1
  (3573, 5490)	1
  (3573, 407)	1
  (3573, 8032)	2
  (3573, 6165)	1
  (3574, 7283)	1
  (3574, 5898)	1
  (3574, 557)	1
  (3574, 5623)	1
  (3574, 3220)	1
  (3574, 8350)	1
  (3574, 5166)	1
      HS
1456   0
4364   0
912    0
1461   1
3389   0
...   ..
4334   0
22     0
3028   0
2402   1
1063   1

[3575 rows x 1 columns]
      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467 

### Paso 2: Ajustando los modelos del clasificador

Utilizaremos el clasificador SVC, debido a que el MLP es incapaz de converger

#### 3.1. Ajustando el modelo para la bolsa de nombres

In [100]:
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report

clasificador_names_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_names_svc.fit(X_names_train, np.ravel(y_names_train))
pred_y_names_svc = clasificador_names_svc.predict(X_names_test)

print("CCR: %f"%(clasificador_names_svc.score(X_names_test, y_names_test)))

print(classification_report(y_names_test, pred_y_names_svc))

CCR: 0.709172
              precision    recall  f1-score   support

           0       0.70      0.88      0.78       526
           1       0.73      0.46      0.57       368

    accuracy                           0.71       894
   macro avg       0.72      0.67      0.67       894
weighted avg       0.71      0.71      0.69       894



#### 3.2. Ajustando el modelo para la bolsa de adjetivos

In [101]:
import numpy as np
from sklearn import svm

clasificador_adj_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_adj_svc.fit(X_adj_train, np.ravel(y_adj_train))
pred_y_adj_svc = clasificador_adj_svc.predict(X_adj_test)

print("CCR: %f"%(clasificador_adj_svc.score(X_adj_test, y_adj_test)))

print(classification_report(y_adj_test, pred_y_adj_svc))

CCR: 0.626398
              precision    recall  f1-score   support

           0       0.62      0.93      0.75       526
           1       0.66      0.19      0.30       368

    accuracy                           0.63       894
   macro avg       0.64      0.56      0.52       894
weighted avg       0.64      0.63      0.56       894



#### 3.3. Ajustando el modelo para la bolsa de adverbios

In [102]:
import numpy as np
from sklearn import svm

clasificador_adv_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_adv_svc.fit(X_adv_train, np.ravel(y_adv_train))
pred_y_adv_svc = clasificador_adv_svc.predict(X_adv_test)

print("CCR: %f"%(clasificador_adv_svc.score(X_adv_test, y_adv_test)))
print(classification_report(y_adv_test, pred_y_adv_svc))

CCR: 0.572707
              precision    recall  f1-score   support

           0       0.59      0.92      0.72       526
           1       0.41      0.08      0.14       368

    accuracy                           0.57       894
   macro avg       0.50      0.50      0.43       894
weighted avg       0.51      0.57      0.48       894



#### 3.4. Ajustando el modelo para la bolsa de nombres y adjetivos

In [103]:
import numpy as np
from sklearn import svm

clasificador_nameadj_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_nameadj_svc.fit(X_nameadj_train, np.ravel(y_nameadj_train))
pred_y_nameadj_svc = clasificador_nameadj_svc.predict(X_nameadj_test)

print("CCR: %f"%(clasificador_nameadj_svc.score(X_nameadj_test, y_nameadj_test)))
print(classification_report(y_nameadj_test, pred_y_nameadj_svc))

CCR: 0.717002
              precision    recall  f1-score   support

           0       0.71      0.86      0.78       526
           1       0.72      0.51      0.60       368

    accuracy                           0.72       894
   macro avg       0.72      0.69      0.69       894
weighted avg       0.72      0.72      0.71       894



#### 3.5. Ajustando el modelo para la bolsa de adjetivos y adverbios

In [104]:
import numpy as np
from sklearn import svm

clasificador_adjadv_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_adjadv_svc.fit(X_adjadv_train, np.ravel(y_adjadv_train))
pred_y_adjadv_svc = clasificador_adjadv_svc.predict(X_adjadv_test)

print("CCR: %f"%(clasificador_adjadv_svc.score(X_adjadv_test, y_adjadv_test)))
print(classification_report(y_adjadv_test, pred_y_adjadv_svc))

CCR: 0.624161
              precision    recall  f1-score   support

           0       0.62      0.91      0.74       526
           1       0.63      0.21      0.31       368

    accuracy                           0.62       894
   macro avg       0.63      0.56      0.53       894
weighted avg       0.63      0.62      0.57       894



#### 3.6. Ajustando el modelo para la bolsa de nombres y adverbios

In [105]:
import numpy as np
from sklearn import svm

clasificador_nameadv_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_nameadv_svc.fit(X_nameadv_train, np.ravel(y_nameadv_train))
pred_y_nameadv_svc = clasificador_nameadv_svc.predict(X_nameadv_test)

print("CCR: %f"%(clasificador_nameadv_svc.score(X_nameadv_test, y_nameadv_test)))
print(classification_report(y_nameadv_test, pred_y_nameadv_svc))

CCR: 0.706935
              precision    recall  f1-score   support

           0       0.70      0.87      0.78       526
           1       0.72      0.47      0.57       368

    accuracy                           0.71       894
   macro avg       0.71      0.67      0.67       894
weighted avg       0.71      0.71      0.69       894



#### 3.7. Ajustando el modelo para la bolsa de nombres, adjetivos y adverbios

In [106]:
import numpy as np
from sklearn import svm

clasificador_nameadjadv_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_nameadjadv_svc.fit(X_nameadjadv_train, np.ravel(y_nameadjadv_train))
pred_y_nameadjadv_svc = clasificador_nameadjadv_svc.predict(X_nameadjadv_test)

print("CCR: %f"%(clasificador_nameadjadv_svc.score(X_nameadjadv_test, y_nameadjadv_test)))
print(classification_report(y_nameadjadv_test, pred_y_nameadjadv_svc))

CCR: 0.723714
              precision    recall  f1-score   support

           0       0.72      0.87      0.79       526
           1       0.73      0.52      0.61       368

    accuracy                           0.72       894
   macro avg       0.73      0.69      0.70       894
weighted avg       0.73      0.72      0.71       894



## Probando los modelos en datos reales

Una vez convergidos los modelos, los aplicamos sobre datos reales, utilizando para ello el conjunto de evaluación (eval)

### 3.1. Prueba con solo nombres

In [124]:
eval_names = onlynames_vectorizer_train.transform(corpus_onlynames_str_eval['text'])
evalPredict_names_svc = clasificador_names_svc.predict(eval_names)

print("CCR: %f"%(clasificador_names_svc.score(eval_names, labels_eval)))
print(classification_report(labels_eval, evalPredict_names_svc))
print(evalPredict_names_svc)

CCR: 0.680000
              precision    recall  f1-score   support

           0       0.62      0.92      0.74        50
           1       0.85      0.44      0.58        50

    accuracy                           0.68       100
   macro avg       0.73      0.68      0.66       100
weighted avg       0.73      0.68      0.66       100

[0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1
 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 3.2. Prueba con solo adjetivos

In [125]:
eval_adjectives = onlyadjectives_vectorizer_train.transform(corpus_onlyadjectives_str_eval['text'])
evalPredict_adj_svc = clasificador_adj_svc.predict(eval_adjectives)

print("CCR: %f"%(clasificador_adj_svc.score(eval_adjectives, labels_eval)))
print(classification_report(labels_eval, evalPredict_adj_svc))
print(evalPredict_adj_svc)

CCR: 0.610000
              precision    recall  f1-score   support

           0       0.56      0.98      0.72        50
           1       0.92      0.24      0.38        50

    accuracy                           0.61       100
   macro avg       0.74      0.61      0.55       100
weighted avg       0.74      0.61      0.55       100

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 3.3. Prueba con solo adverbios

In [127]:
eval_adv = onlyadverbs_vectorizer_train.transform(corpus_onlyadverbs_str_eval['text'])
evalPredict_adv_svc = clasificador_adv_svc.predict(eval_adv)

print("CCR: %f"%(clasificador_adv_svc.score(eval_adv, labels_eval)))
print(classification_report(labels_eval, evalPredict_adv_svc))
print(evalPredict_adv_svc)

CCR: 0.550000
              precision    recall  f1-score   support

           0       0.53      0.96      0.68        50
           1       0.78      0.14      0.24        50

    accuracy                           0.55       100
   macro avg       0.65      0.55      0.46       100
weighted avg       0.65      0.55      0.46       100

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


### 3.4. Prueba con nombres y adjetivos

In [134]:
eval_nameadj = nameadj_vectorizer_train.transform(corpus_nameadj_str_eval['text'])
evalPredict_nameadj_svc = clasificador_nameadj_svc.predict(eval_nameadj)

print("CCR: %f"%(clasificador_nameadj_svc.score(eval_nameadj, labels_eval)))
print(classification_report(labels_eval, evalPredict_nameadj_svc))
print(evalPredict_nameadj_svc)

CCR: 0.710000
              precision    recall  f1-score   support

           0       0.65      0.92      0.76        50
           1       0.86      0.50      0.63        50

    accuracy                           0.71       100
   macro avg       0.75      0.71      0.70       100
weighted avg       0.75      0.71      0.70       100

[0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1
 0 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 3.5. Prueba con adverbios y adjetivos

In [136]:
eval_adjadv = adjadv_vectorizer_train.transform(corpus_adjadv_str_eval['text'])
evalPredict_adjadv_svc = clasificador_adjadv_svc.predict(eval_adjadv)

print("CCR: %f"%(clasificador_adjadv_svc.score(eval_adjadv, labels_eval)))
print(classification_report(labels_eval, evalPredict_adjadv_svc))
print(evalPredict_adjadv_svc)

CCR: 0.620000
              precision    recall  f1-score   support

           0       0.57      0.98      0.72        50
           1       0.93      0.26      0.41        50

    accuracy                           0.62       100
   macro avg       0.75      0.62      0.56       100
weighted avg       0.75      0.62      0.56       100

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 3.6. Prueba con nombres y adverbios

In [138]:
eval_nameadv = nameadv_vectorizer_train.transform(corpus_nameadv_str_eval['text'])
evalPredict_nameadv_svc = clasificador_nameadv_svc.predict(eval_nameadv)

print("CCR: %f"%(clasificador_nameadv_svc.score(eval_nameadv, labels_eval)))
print(classification_report(labels_eval, evalPredict_nameadv_svc))
print(evalPredict_nameadv_svc)

CCR: 0.720000
              precision    recall  f1-score   support

           0       0.66      0.92      0.77        50
           1       0.87      0.52      0.65        50

    accuracy                           0.72       100
   macro avg       0.76      0.72      0.71       100
weighted avg       0.76      0.72      0.71       100

[0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1
 0 1 1 1 1 0 0 0 1 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### 3.7. Prueba con nombres, adjetivos y adverbios

In [141]:
eval_nameadjadv = nameadjadv_vectorizer_train.transform(corpus_nameadjadv_str_eval['text'])
evalPredict_nameadjadv_svc = clasificador_nameadjadv_svc.predict(eval_nameadjadv)

print("CCR: %f"%(clasificador_nameadjadv_svc.score(eval_nameadjadv, labels_eval)))
print(classification_report(labels_eval, evalPredict_nameadjadv_svc))
print(evalPredict_nameadjadv_svc)

CCR: 0.740000
              precision    recall  f1-score   support

           0       0.68      0.92      0.78        50
           1       0.88      0.56      0.68        50

    accuracy                           0.74       100
   macro avg       0.78      0.74      0.73       100
weighted avg       0.78      0.74      0.73       100

[0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1
 0 1 1 1 1 1 0 0 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Conclusiones

El TweetTokenizer es una gran herramienta para extraer tokens de mensajes provenientes de Twitter. Mejora la identificación de tokens con características especiales, como hashtags, menciones, URLs... Esto permite realizar un filtrado posterior de forma mas precisa, sin necesidad de recurrir a expresiones regulares.

Respecto al POS Tagging, hemos descubierto que es una herramienta bastante poderosa para realizar un análisis gramatical. Por contra, la clasificación gramatical no es perfecta, y el tiempo de cómputo es bastante alto.

En el uso de la categorización para conteo de categorías, no se aprecia ningún patrón claro que permita afirmar que el exceso de una o varias categorías determine si el texto es de odio o no lo es. 

Al utilizarlo para filtrar categorías en el bag of words, y entrenar los modelos de clasificación con los diferentes grupos de categorías, observamos que los mejores resultados se obtienen con el conjunto de nombres, adjetivos y adverbios (78% de precisión); y que los peores resultados se dan con el conjunto de solo adverbios (65% de precisión).

El filtrado de categorías gramaticales también ha requerido un gran tiempo de cómputo, de hasta hora y media en algunos filtrados.